In [39]:
import os
import cv2
import shutil
import numpy as np
from PIL import Image

images = '../images/gbif_images'
save_dir = "../data/to_edit"
image_save_dir = "../data/dataset/images"
label_save_dir = "../data/dataset/labels"

os.makedirs(save_dir, exist_ok=True)
os.makedirs(image_save_dir, exist_ok=True)
os.makedirs(label_save_dir, exist_ok=True)

Create masks to edit and move images to dataset folder

In [12]:
for image in os.listdir(images):
    # Load the image in grayscale mode
    orig = cv2.imread(os.path.join(images, image), cv2.IMREAD_GRAYSCALE)

    # Apply Otsu's thresholding to segment the image
    ret, thresh = cv2.threshold(orig, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Create the masks using numpy indexing
    mask1 = np.where(thresh == 0, 0, 255).astype('uint8')  # Foreground mask

    # Convert the masks to images and save
    Image.fromarray(mask1).save(os.path.join(save_dir, os.path.basename(image).replace('.jpg', '_mask.jpg')), "JPEG")
    
    # copy the original image to the image_save_dir
    shutil.copy(os.path.join(images, image), os.path.join(image_save_dir, os.path.basename(image)))

Now manually edit using photo editing tools (remove the non masked white parts and connect the parts of the plant where the tape was)

Generate the segmentation points on the edited foreground picture

In [40]:
os.makedirs('./test', exist_ok=True)

for idx, mask in enumerate(os.listdir(save_dir)):
    image = Image.open(os.path.join(save_dir, mask)).convert("L")
    image_width, image_height = image.size
    image_array = np.array(image)
    
    binary_mask = image_array > 128
    image_array_uint8 = np.uint8(binary_mask * 255)
    
    contours, _ = cv2.findContours(image_array_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    contours = [contour for contour in contours if cv2.contourArea(contour) > 100]
    
    # for testing purposes
    image_bgr = cv2.cvtColor(image_array_uint8, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(image_bgr, contours, -1, (255, 0, 0), 4)
    contour_image = Image.fromarray(image_bgr)
    contour_image.save(f'./test/{mask}')
    
    with open(os.path.join(label_save_dir, mask.replace('_mask.jpg', '.txt')), "w") as f:
        for contour in contours:
            normalized_contour_points = contour[:, 0, :] / [image_width, image_height]
            normalized_contour_xyxyxy_format = normalized_contour_points.flatten().tolist()
            f.write('0 ')
            f.write(" ".join(map(str, normalized_contour_xyxyxy_format)))
            f.write('\n')